In [1]:
#Import the necessary modules
import pandas as pd
import numpy as np

#Metrics library
from sklearn.metrics import mean_squared_log_error

#Preprocessing library
from sklearn.preprocessing import StandardScaler
import seaborn as sns

#Pipeline library
#from sklearn.pipeline import Pipeline
from sklearn.pipeline import Pipeline
#Algorithm libraries
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.svm import SVR
from keras.models import Sequential
from keras.layers import Dense

2022-12-16 11:53:19.803206: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-16 11:53:23.807481: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-16 11:53:23.807522: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-16 11:53:31.099334: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

In [11]:
#Load the data from file
data=pd.read_csv('../data/train.csv')


In [12]:
#Find the unique values
data['AdoptionSpeed'].unique()

array([2, 0, 3, 1, 4])

In [13]:
#column names
data.columns

Index(['Type', 'Name', 'Age', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
       'Sterilized', 'Health', 'Quantity', 'Fee', 'State', 'RescuerID',
       'VideoAmt', 'Description', 'PetID', 'PhotoAmt', 'AdoptionSpeed'],
      dtype='object')

In [14]:
data.corr() #columns are not highly positive or negative correlated so we can retain all the fields
#since i want to create a light weight model removing some unwanted fields like color, maturity size,FurLength,Vaccinated,Dewormed,Sterilized,name, petid, rescuerid, video amt, photo amt .

/tmp/ipykernel_5897/3583392846.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  data.corr() #columns are not highly positive or negative correlated so we can retain all the fields


,Type,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,FurLength,Vaccinated,Dewormed,Sterilized,Health,Quantity,Fee,State,VideoAmt,PhotoAmt,AdoptionSpeed
Type,1.000000,-0.147162,0.057724,-0.043568,0.060843,0.097680,0.246333,0.201693,-0.171811,0.003036,0.102907,0.025508,0.006737,-0.006864,0.036423,-0.042134,0.124549,-0.005967,0.049368,-0.091240
Age,-0.147162,1.000000,-0.314346,-0.041010,-0.123423,0.090378,-0.039427,-0.053546,0.093673,0.153092,-0.136061,-0.053360,-0.189450,0.103215,-0.113076,0.089168,0.024523,-0.015969,-0.084778,0.100510
Breed1,0.057724,-0.314346,1.000000,-0.157053,0.071321,-0.035510,-0.007081,-0.000682,-0.010507,-0.108059,0.046518,0.013216,0.052698,-0.034893,0.090194,-0.189526,-0.026832,0.020626,0.043395,0.107834
Breed2,-0.043568,-0.041010,-0.157053,1.000000,0.056907,-0.020324,0.003015,0.035679,0.049879,0.106220,0.005638,-0.009817,-0.005989,-0.029285,0.038125,0.013508,-0.037993,0.001569,0.045953,-0.018642
Gender,0.060843,-0.123423,0.071321,0.056907,1.000000,-0.115351,0.026507,0.259309,-0.091819,-0.030404,0.078702,0.091431,0.040645,-0.045177,0.494489,-0.050491,0.002564,0.016816,0.098435,0.057622
Color1,0.097680,0.090378,-0.035510,-0.020324,-0.115351,1.000000,-0.113290,-0.282897,-0.029468,0.066967,-0.017475,-0.019318,-0.037050,0.024410,-0.116688,0.054180,0.024012,-0.006435,-0.041323,-0.044192
Color2,0.246333,-0.039427,-0.007081,0.003015,0.026507,-0.113290,1.000000,0.088742,-0.065155,-0.008764,0.026193,0.005812,0.007370,-0.004307,0.019876,-0.017308,0.026944,0.023983,0.059113,-0.038579
Color3,0.201693,-0.053546,-0.000682,0.035679,0.259309,-0.282897,0.088742,1.000000,-0.051730,0.013581,0.054053,0.048617,0.038899,-0.024782,0.272548,-0.019839,0.010554,0.019526,0.103662,-0.007014
MaturitySize,-0.171811,0.093673,-0.010507,0.049879,-0.091819,-0.029468,-0.065155,-0.051730,1.000000,0.095142,-0.087722,-0.067352,-0.066321,-0.012957,-0.042832,0.041144,-0.062584,0.017032,0.015593,0.045653
FurLength,0.003036,0.153092,-0.108059,0.106220,-0.030404,0.066967,-0.008764,0.013581,0.095142,1.000000,-0.006010,0.016463,0.032092,0.028096,-0.038388,0.159693,-0.029317,-0.014740,-0.031557,-0.091337


In [15]:
#Extracting the needed columns
data1=data[['Type', 'Age', 'Breed1', 'Breed2', 'Gender', 'Health', 'Quantity', 'Fee','AdoptionSpeed']]

In [16]:
#New dataframe
data1

,Type,Age,Breed1,Breed2,Gender,Health,Quantity,Fee,AdoptionSpeed
0,2,3,299,0,1,1,1,100,2
1,2,1,265,0,1,1,1,0,0
2,1,1,307,0,1,1,1,0,3
3,1,4,307,0,2,1,1,150,2
4,1,1,307,0,1,1,1,0,2
...,...,...,...,...,...,...,...,...,...
14988,2,2,266,0,3,1,4,0,2
14989,2,60,265,264,3,1,2,0,4
14990,2,2,265,266,3,1,5,30,3
14991,2,9,266,0,2,1,1,0,4


In [17]:
#Split X and Y data
x=data1.drop(labels='AdoptionSpeed', axis=1)
y= data['AdoptionSpeed']

In [18]:
#Splitting the train and test datasets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [19]:
#Computing root mean squared log error
def compute_rmsle(y_test: np.ndarray, y_pred: np.ndarray, precision: int = 2) -> float:
    rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))
    return round(rmsle, precision)

In [20]:
#Creating pipeline for standard scaler and support vector regressor
#Categorical columns are in numeric form so we dont need encoding
pipe = Pipeline(steps=[('scaler',StandardScaler()),
                       ('svr',SVR(kernel = 'rbf'))])

#Creating the model on Training Data
pipe.fit(X_train, y_train)

#Predict the output
y_pred = pipe.predict(X_test)

#Computing the error
svr_rmse=compute_rmsle(y_test,y_pred)
print(svr_rmse)

0.38


In [21]:
pipe = Pipeline(steps=[('scaler',StandardScaler()),
                        ('xgbr',XGBRegressor(max_depth=3, learning_rate=0.01, n_estimators=200, objective='reg:linear', booster='gbtree')),
                        ])

#Creating the model on Training Data
pipe.fit(X_train, y_train)

#Predict the output
y_pred = pipe.predict(X_test)

#Computing the error
xgbr_rmse=compute_rmsle(y_test,y_pred)
print(xgbr_rmse)

[12:11:24] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
0.37


In [22]:
pipe = Pipeline(steps=[('scaler',StandardScaler()),
                       ('rf',RandomForestRegressor(n_estimators=20,max_depth=10,criterion='mse')),
                        ])
 
#Creating the model on Training Data
pipe.fit(X_train, y_train)

#Predict the output
y_pred = pipe.predict(X_test)

#Computing the error
rf_rmse=compute_rmsle(y_test,y_pred)
print(rf_rmse)

/home/codespace/.local/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(


0.37


In [23]:
#Artificial neural network
 
# create ANN model
model_nn = Sequential()
 
# Defining the Input layer and FIRST hidden layer, both are same!
model_nn.add(Dense(units=5, input_dim=8, kernel_initializer='normal', activation='relu'))
 
# Defining the Second layer of the model
# after the first layer we don't have to specify input_dim as keras configure it automatically
model_nn.add(Dense(units=5, kernel_initializer='normal', activation='tanh'))
 
# The output neuron is a single fully connected node 
# Since we will be predicting a single number
model_nn.add(Dense(1, kernel_initializer='normal'))
 
# Compiling the model
model_nn.compile(loss='mean_squared_error', optimizer='adam')
 
# Fitting the ANN to the Training set
model_nn.fit(X_train, y_train ,batch_size = 20, epochs = 50, verbose=1)

2022-12-16 12:11:42.738621: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-12-16 12:11:42.749256: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2022-12-16 12:11:42.749332: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (codespaces-28ce4f): /proc/driver/nvidia/version does not exist
2022-12-16 12:11:42.866139: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/50
600/600 [==============================] - 3s 2ms/step - loss: 2.7231
Epoch 2/50
600/600 [==============================] - 1s 2ms/step - loss: 1.3831
Epoch 3/50
600/600 [==============================] - 1s 2ms/step - loss: 1.3727
Epoch 4/50
600/600 [==============================] - 1s 2ms/step - loss: 1.3599
Epoch 5/50
600/600 [==============================] - 1s 2ms/step - loss: 1.3423
Epoch 6/50
600/600 [==============================] - 1s 2ms/step - loss: 1.3284
Epoch 7/50
600/600 [==============================] - 1s 1ms/step - loss: 1.3101
Epoch 8/50
600/600 [==============================] - 1s 2ms/step - loss: 1.2904
Epoch 9/50
600/600 [==============================] - 1s 2ms/step - loss: 1.2839
Epoch 10/50
600/600 [==============================] - 1s 1ms/step - loss: 1.2801
Epoch 11/50
600/600 [==============================] - 1s 2ms/step - loss: 1.2723
Epoch 12/50
600/600 [==============================] - 1s 1ms/step - loss: 1.2716
Epoch 13/50
600/600 [====

In [24]:
#Predicting the output
y_pred_nn = model_nn.predict(X_test)

94/94 [==============================] - 0s 2ms/step


In [25]:
y_pred_nn

array([[3.0945175],
       [2.050024 ],
       [2.6453013],
       ...,
       [2.4209228],
       [2.9350371],
       [2.0040264]], dtype=float32)

In [26]:
#Computing the error
compute_rmsle(y_test,y_pred_nn)

0.37